# Workflow Human-in-the-Loop con Microsoft Agent Framework

## 🎯 Obiettivi di apprendimento

In questo notebook, imparerai a implementare workflow **human-in-the-loop** utilizzando il `RequestInfoExecutor` del Microsoft Agent Framework. Questo potente modello consente di mettere in pausa i workflow AI per raccogliere input umani, rendendo gli agenti interattivi e dando agli esseri umani il controllo sulle decisioni critiche.

## 🔄 Cos'è Human-in-the-Loop?

**Human-in-the-loop (HITL)** è un modello di progettazione in cui gli agenti AI interrompono l'esecuzione per richiedere input umano prima di continuare. Questo è essenziale per:

- ✅ **Decisioni critiche** - Ottenere l'approvazione umana prima di intraprendere azioni importanti
- ✅ **Situazioni ambigue** - Permettere agli esseri umani di chiarire quando l'AI è incerta
- ✅ **Preferenze dell'utente** - Chiedere agli utenti di scegliere tra opzioni multiple
- ✅ **Conformità e sicurezza** - Garantire la supervisione umana per operazioni regolamentate
- ✅ **Esperienze interattive** - Creare agenti conversazionali che rispondano agli input degli utenti

## 🏗️ Come funziona nel Microsoft Agent Framework

Il framework fornisce tre componenti chiave per HITL:

1. **`RequestInfoExecutor`** - Un executor speciale che mette in pausa il workflow ed emette un `RequestInfoEvent`
2. **`RequestInfoMessage`** - Classe base per i payload di richiesta tipizzati inviati agli esseri umani
3. **`RequestResponse`** - Correla le risposte umane con le richieste originali utilizzando `request_id`

**Modello di workflow:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Il nostro esempio: Prenotazione di hotel con conferma utente

Costruiremo sul workflow condizionale aggiungendo una conferma umana **prima** di suggerire destinazioni alternative:

1. L'utente richiede una destinazione (es. "Parigi")
2. L'`availability_agent` verifica se ci sono camere disponibili
3. **Se non ci sono camere** → Il `confirmation_agent` chiede "Vuoi vedere alternative?"
4. Il workflow **si mette in pausa** utilizzando `RequestInfoExecutor`
5. **L'umano risponde** "sì" o "no" tramite input console
6. Il `decision_manager` instrada in base alla risposta:
   - **Sì** → Mostra destinazioni alternative
   - **No** → Annulla la richiesta di prenotazione
7. Mostra il risultato finale

Questo dimostra come dare agli utenti il controllo sulle proposte dell'agente!

---

Iniziamo! 🚀


## Passaggio 1: Importare le librerie necessarie

Importiamo i componenti standard del Framework Agent oltre alle **classi specifiche per l'intervento umano**:
- `RequestInfoExecutor` - Esecutore che mette in pausa il flusso di lavoro per l'input umano
- `RequestInfoEvent` - Evento emesso quando viene richiesto un input umano
- `RequestInfoMessage` - Classe base per i payload di richiesta tipizzati
- `RequestResponse` - Correlazione tra le risposte umane e le richieste
- `WorkflowOutputEvent` - Evento per rilevare gli output del flusso di lavoro


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Passaggio 2: Definire i modelli Pydantic per output strutturati

Questi modelli definiscono lo **schema** che gli agenti restituiranno. Manteniamo tutti i modelli dal flusso di lavoro condizionale e aggiungiamo:

**Novità per Human-in-the-Loop:**
- `HumanFeedbackRequest` - Sottoclasse di `RequestInfoMessage` che definisce il payload della richiesta inviato agli esseri umani
  - Contiene `prompt` (domanda da porre) e `destination` (contesto sulla città non disponibile)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Passaggio 3: Crea lo Strumento di Prenotazione Hotel

Lo stesso strumento del flusso di lavoro condizionale - verifica se ci sono camere disponibili nella destinazione.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Passaggio 4: Definire le Funzioni di Condizione per il Routing

Abbiamo bisogno di **quattro funzioni di condizione** per il nostro flusso di lavoro con intervento umano:

**Dal flusso di lavoro condizionale:**
1. `has_availability_condition` - Instrada quando gli hotel SONO disponibili
2. `no_availability_condition` - Instrada quando gli hotel NON sono disponibili

**Nuove per l'intervento umano:**
3. `user_wants_alternatives_condition` - Instrada quando l'utente dice "sì" alle alternative
4. `user_declines_alternatives_condition` - Instrada quando l'utente dice "no" alle alternative


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Passaggio 5: Creare l'Esecutore del Decision Manager

Questo è il **cuore del modello human-in-the-loop**! Il `DecisionManager` è un `Executor` personalizzato che:

1. **Riceve il feedback umano** tramite oggetti `RequestResponse`
2. **Elabora la decisione dell'utente** (sì/no)
3. **Instrada il flusso di lavoro** inviando messaggi agli agenti appropriati

Caratteristiche principali:
- Utilizza il decoratore `@handler` per esporre i metodi come passaggi del flusso di lavoro
- Riceve `RequestResponse[HumanFeedbackRequest, str]` contenente sia la richiesta originale che la risposta dell'utente
- Genera semplici messaggi "sì" o "no" che attivano le nostre funzioni condizionali


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Passaggio 6: Creare un Executor di Visualizzazione Personalizzato

Lo stesso executor di visualizzazione del flusso di lavoro condizionale - produce i risultati finali come output del flusso di lavoro.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Passaggio 7: Caricare le Variabili d'Ambiente

Configura il client LLM (GitHub Models, Azure OpenAI o OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Passaggio 8: Creare Agenti AI ed Esecutori

Creiamo **sei componenti del flusso di lavoro**:

**Agenti (incapsulati in AgentExecutor):**
1. **availability_agent** - Verifica la disponibilità degli hotel utilizzando lo strumento
2. **confirmation_agent** - 🆕 Prepara la richiesta di conferma umana
3. **alternative_agent** - Suggerisce città alternative (quando l'utente dice sì)
4. **booking_agent** - Incoraggia la prenotazione (quando le camere sono disponibili)
5. **cancellation_agent** - 🆕 Gestisce il messaggio di cancellazione (quando l'utente dice no)

**Esecutori Speciali:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` che mette in pausa il flusso di lavoro per l'input umano
7. **decision_manager** - 🆕 Esecutore personalizzato che instrada in base alla risposta umana (già definito sopra)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Passaggio 9: Costruire il Workflow con Human-in-the-Loop

Ora costruiamo il grafico del workflow con **instradamento condizionale**, includendo il percorso human-in-the-loop:

**Struttura del Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Collegamenti Chiave:**
- `availability_agent → confirmation_agent` (quando non ci sono camere)
- `confirmation_agent → prepare_human_request` (trasformazione tipo)
- `prepare_human_request → request_info_executor` (pausa per intervento umano)
- `request_info_executor → decision_manager` (sempre - fornisce RequestResponse)
- `decision_manager → alternative_agent` (quando l'utente dice "sì")
- `decision_manager → cancellation_agent` (quando l'utente dice "no")
- `availability_agent → booking_agent` (quando ci sono camere disponibili)
- Tutti i percorsi terminano in `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Passaggio 10: Esegui il Caso di Test 1 - Città SENZA Disponibilità (Parigi con Conferma Umana)

Questo test dimostra il **ciclo completo con intervento umano**:

1. Richiedi un hotel a Parigi
2. availability_agent verifica → Nessuna camera disponibile
3. confirmation_agent crea una domanda per l'interazione umana
4. request_info_executor **mette in pausa il flusso di lavoro** ed emette `RequestInfoEvent`
5. **L'applicazione rileva l'evento e chiede all'utente tramite console**
6. L'utente digita "sì" o "no"
7. L'applicazione invia la risposta tramite `send_responses_streaming()`
8. decision_manager instrada in base alla risposta
9. Risultato finale visualizzato

**Schema Chiave:**
- Usa `workflow.run_stream()` per la prima iterazione
- Usa `workflow.send_responses_streaming(pending_responses)` per le iterazioni successive
- Ascolta `RequestInfoEvent` per rilevare quando è necessario l'intervento umano
- Ascolta `WorkflowOutputEvent` per catturare i risultati finali


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Passo 11: Esegui il Caso di Test 2 - Città CON Disponibilità (Stoccolma - Nessun Intervento Umano Necessario)

Questo test dimostra il **percorso diretto** quando le camere sono disponibili:

1. Richiedi un hotel a Stoccolma
2. availability_agent verifica → Camere disponibili ✅
3. booking_agent suggerisce la prenotazione
4. display_result mostra la conferma
5. **Nessun intervento umano richiesto!**

Il flusso di lavoro salta completamente il percorso con intervento umano quando le camere sono disponibili.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Punti Chiave e Migliori Pratiche per il Coinvolgimento Umano

### ✅ Cosa Hai Imparato:

#### 1. **Pattern RequestInfoExecutor**
Il pattern di coinvolgimento umano nel Microsoft Agent Framework utilizza tre componenti chiave:
- `RequestInfoExecutor` - Mette in pausa il flusso di lavoro ed emette eventi
- `RequestInfoMessage` - Classe base per i payload delle richieste tipizzate (sottoclassa questa!)
- `RequestResponse` - Correla le risposte umane con le richieste originali

**Comprensione Critica:**
- `RequestInfoExecutor` NON raccoglie input direttamente - si limita a mettere in pausa il flusso di lavoro
- Il codice della tua applicazione deve ascoltare `RequestInfoEvent` e raccogliere input
- Devi chiamare `send_responses_streaming()` con un dizionario che mappa `request_id` alla risposta dell'utente

#### 2. **Pattern di Esecuzione Streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Architettura Basata su Eventi**
Ascolta eventi specifici per controllare il flusso di lavoro:
- `RequestInfoEvent` - È necessario un input umano (flusso di lavoro in pausa)
- `WorkflowOutputEvent` - Il risultato finale è disponibile (flusso di lavoro completato)
- `WorkflowStatusEvent` - Cambiamenti di stato (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, ecc.)

#### 4. **Esecutori Personalizzati con @handler**
Il `DecisionManager` dimostra come creare esecutori che:
- Utilizzano il decoratore `@handler` per esporre metodi come passaggi del flusso di lavoro
- Ricevono messaggi tipizzati (es. `RequestResponse[HumanFeedbackRequest, str]`)
- Instradano il flusso di lavoro inviando messaggi ad altri esecutori
- Accedono al contesto tramite `WorkflowContext`

#### 5. **Instradamento Condizionale con Decisioni Umane**
Puoi creare funzioni condizionali che valutano le risposte umane:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Applicazioni Reali:

1. **Flussi di Lavoro di Approvazione**
   - Ottieni l'approvazione del manager prima di elaborare i report di spesa
   - Richiedi una revisione umana prima di inviare email automatiche
   - Conferma transazioni di alto valore prima dell'esecuzione

2. **Moderazione dei Contenuti**
   - Segnala contenuti discutibili per revisione umana
   - Chiedi ai moderatori di prendere decisioni finali su casi limite
   - Escalare agli umani quando la fiducia dell'IA è bassa

3. **Servizio Clienti**
   - Lascia che l'IA gestisca automaticamente le domande di routine
   - Escalare problemi complessi agli agenti umani
   - Chiedi al cliente se desidera parlare con un umano

4. **Elaborazione Dati**
   - Chiedi agli umani di risolvere voci di dati ambigue
   - Conferma interpretazioni dell'IA di documenti poco chiari
   - Lascia che gli utenti scelgano tra interpretazioni valide multiple

5. **Sistemi Critici per la Sicurezza**
   - Richiedi conferma umana prima di azioni irreversibili
   - Ottieni approvazione prima di accedere a dati sensibili
   - Conferma decisioni in settori regolamentati (sanità, finanza)

6. **Agenti Interattivi**
   - Crea bot conversazionali che pongono domande di approfondimento
   - Crea assistenti che guidano gli utenti attraverso processi complessi
   - Progetta agenti che collaborano con gli umani passo dopo passo

### 🔄 Confronto: Con vs Senza Coinvolgimento Umano

| Caratteristica | Flusso di Lavoro Condizionale | Flusso di Lavoro con Coinvolgimento Umano |
|----------------|-------------------------------|------------------------------------------|
| **Esecuzione** | Singolo `workflow.run()` | Ciclo con `run_stream()` + `send_responses_streaming()` |
| **Input Utente** | Nessuno (completamente automatizzato) | Prompt interattivi tramite `input()` o UI |
| **Componenti** | Agenti + Esecutori | + RequestInfoExecutor + DecisionManager |
| **Eventi** | Solo AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, ecc. |
| **Pausa** | Nessuna pausa | Il flusso di lavoro si ferma a RequestInfoExecutor |
| **Controllo Umano** | Nessun controllo umano | Gli umani prendono decisioni chiave |
| **Caso d'Uso** | Automazione | Collaborazione e supervisione |

### 🚀 Pattern Avanzati:

#### Punti di Decisione Umana Multipli
Puoi avere più nodi `RequestInfoExecutor` nello stesso flusso di lavoro:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Gestione dei Timeout
Implementa timeout per le risposte umane:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrazione con UI Avanzata
Invece di `input()`, integra con UI web, Slack, Teams, ecc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Coinvolgimento Umano Condizionale
Richiedi input umano solo in situazioni specifiche:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Migliori Pratiche:

1. **Sottoclassa Sempre RequestInfoMessage**
   - Fornisce sicurezza di tipo e validazione
   - Abilita un contesto ricco per il rendering della UI
   - Chiarisce l'intento di ogni tipo di richiesta

2. **Usa Prompt Descrittivi**
   - Includi il contesto su ciò che stai chiedendo
   - Spiega le conseguenze di ogni scelta
   - Mantieni le domande semplici e chiare

3. **Gestisci Input Inaspettati**
   - Valida le risposte degli utenti
   - Fornisci valori predefiniti per input non validi
   - Dai messaggi di errore chiari

4. **Traccia gli ID delle Richieste**
   - Usa la correlazione tra request_id e risposte
   - Non cercare di gestire lo stato manualmente

5. **Progetta per Non Bloccare**
   - Non bloccare i thread in attesa di input
   - Usa pattern asincroni ovunque
   - Supporta istanze concorrenti del flusso di lavoro

### 📚 Concetti Correlati:

- **Middleware degli Agenti** - Intercetta le chiamate degli agenti (notebook precedente)
- **Gestione dello Stato del Flusso di Lavoro** - Persiste lo stato del flusso di lavoro tra esecuzioni
- **Collaborazione Multi-Agente** - Combina il coinvolgimento umano con team di agenti
- **Architetture Basate su Eventi** - Costruisci sistemi reattivi con eventi

---

### 🎓 Congratulazioni!

Hai padroneggiato i flussi di lavoro con coinvolgimento umano nel Microsoft Agent Framework! Ora sai come:
- ✅ Mettere in pausa i flussi di lavoro per raccogliere input umano
- ✅ Usare RequestInfoExecutor e RequestInfoMessage
- ✅ Gestire l'esecuzione streaming con eventi
- ✅ Creare esecutori personalizzati con @handler
- ✅ Instradare i flussi di lavoro basandoti su decisioni umane
- ✅ Costruire agenti AI interattivi che collaborano con gli umani

**Questo è un pattern fondamentale per costruire sistemi AI affidabili e controllabili!** 🚀



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione AI [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale umana. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
